In [1]:
# Allow `import optiml`. Assumes notbook dir is the standard optiml/notebook dir. 
# No-op if it already in sys.path
import sys
from pathlib import Path
cwd = !pwd
assert cwd[0].endswith("optiml/notebooks")
lib_root = str((Path(cwd[0]) / ".." ).resolve())
if lib_root not in [str(Path(x).resolve()) for x in sys.path]:
    sys.path.append(lib_root)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import optiml.connection as OC

In [2]:
%load_ext autoreload
%autoreload 2
%aimport optiml.queries
%matplotlib inline

OQ = optiml.queries

In [5]:
conn = OC.SnowflakeConnConfig(accountname='VUJSGPR-XG91870',
                              warehousename="XSMALL_WH",
                             ).create_connection() 

qlib = optiml.queries.SnflkQueryLib(conn=conn, dbname="KIV")
sdate, edate = ('2022-01-01', '2022-10-31')

/Users/skumar/.snowsql/config
Connecting...


In [6]:
combined_billing = qlib.get_combined_billing(sdate, 12, 3)

ProgrammingError: 002003 (42S02): SQL compilation error:
Object 'KIV.ACCOUNT_USAGE.PIPE_USAGE_HISTORY' does not exist or not authorized.